In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential , Input
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D, UpSampling2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_crossentropy 
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import  Rescaling, Resizing
import matplotlib.pyplot as plt
import numpy.matlib
import os
import cv2
import pandas as pd
from tensorflow.image import resize
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow_hub as hub
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans , DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import normalize, StandardScaler
from skimage.io import imread , imshow
from scipy import spatial
from sklearn.datasets import make_multilabel_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from mpl_toolkits.mplot3d import Axes3D
from sklearn.multioutput import ClassifierChain
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
import pickle 
import joblib

In [2]:
!gdown --id 11FCdleqCwmHbFqVEKrbZy4QrQMhM2xQW ##Download animals.npy directly from google drive

Downloading...
From: https://drive.google.com/uc?id=11FCdleqCwmHbFqVEKrbZy4QrQMhM2xQW
To: /content/animals.npy
693MB [00:02, 267MB/s]


In [3]:
 ##NOTE for instructors.  I directly loaded file from google drive during testing, but it is not
##possible to share the 'animals.npy' from github since it was more than 600 mb therefore
## fetching file directly from google drive more easy for you to download to this environment
dataset = np.load('animals.npy')  ##Images dataset represented as image tensor
print(dataset.shape)
print("Done!")

(14092, 128, 128, 3)
Done!


In [ ]:
!gdown --id 18CQ8_jInPhWNaiIPEBWPHx35prT9xqlw ## Download img_array directly from google-drive

In [ ]:
img_array = np.load('224_224img.npy') #Rows have the feature vector of 10000 images from dataset.
img_array.shape

In [ ]:
## MobileNet model trained on ImageNet. Directly download from TensorFlow Hub
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
                   trainable=False),  # Can be True, see below.
])
m.build([None, 224, 224, 3])  # Batch input shape.

In [ ]:
## Scale the input data before doing different PCA operations. 
scaled = StandardScaler().fit(img_array)
scaled_array = scaled.transform(img_array)

In [ ]:
pca = PCA(n_components = 30) ##Reduce feature space dimension and cluster its data so that we get a labeling of classes.
pca.fit(scaled_array)
print(f'Total variance of reduced dimension representation is = {round(np.sum(pca.explained_variance_ratio_)*100,3)}%') ##Total variance 
compressed_vector = pca.transform(scaled_array)
print(f'New shape of array = {compressed_vector.shape}')
kmeans = KMeans(n_clusters= 5 ,n_jobs=-1, random_state=22, max_iter= 500)
kmeans.fit(compressed_vector)
print('Done!')

In [ ]:
##To visualize the data, reduce the dimension of input to 3. Visualize below
pca_3d = PCA(n_components = 3)
pca_3d.fit(scaled_array)
feature_vector = pca_3d.transform(scaled_array)
kmeans_10k = KMeans(n_clusters= 5 ,n_jobs=-1, random_state=22, max_iter= 500)
kmeans_10k.fit(feature_vector)

In [ ]:
fig = plt.figure(figsize=(10, 6))
# Create 3D container of 10.000 Data Point
ax = plt.axes(projection = '3d')
# Visualize 3D scatter plot
v = np.zeros((10000,4)) ##Visualize 10000 images in the image array in 3D feature space.
v[:,:3] = feature_vector
v[:,3] = kmeans_10k.labels_
df = pd.DataFrame(v, columns=['Feature1', 'Feature2','Feature3', "Cluster"])
ax.scatter(feature_vector[:,0],feature_vector[:,1],feature_vector[:,2], marker="o", c=df["Cluster"], s=40, cmap="RdBu")
# Give labels
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
# Save figure
plt.savefig('3d_scatter.png', dpi = 400);

In [ ]:
##Soft K-Means Clustering Function
def soft_clustering_weights(data, cluster_centres, **kwargs):
    
    """
    Function to calculate the weights from soft k-means
    data: Array of data. shape = N x F, for N data points and F Features
    cluster_centres: Array of cluster centres. shape = Nc x F, for Nc number of clusters. Input kmeans.cluster_centres_ directly.
    param: m - keyword argument, fuzziness of the clustering. Default 2
    """
    
    # Fuzziness parameter m>=1. Where m=1 => hard segmentation
    m = 2
    if 'm' in kwargs:
        m = kwargs['m']
    
    Nclusters = cluster_centres.shape[0]
    Ndp = data.shape[0]
    Nfeatures = data.shape[1]

    # Get distances from the cluster centres for each data point and each cluster
    EuclidDist = np.zeros((Ndp, Nclusters))
    for i in range(Nclusters):
        EuclidDist[:,i] = np.sum((data-numpy.matlib.repmat(cluster_centres[i], Ndp, 1))**2,axis=1)

    
    # Denominator of the weight from wikipedia:
    invWeight = EuclidDist**(2/(m-1))*numpy.matlib.repmat(np.sum((1./EuclidDist)**(2/(m-1)),axis=1).reshape(-1,1),1,Nclusters)
    Weight = 1./invWeight
    
    return Weight

In [ ]:
weights = soft_clustering_weights(compressed_vector,kmeans.cluster_centers_)

In [ ]:
animal_dict = {3 : "Butterfly", 4 : "Horse", 1 : "Elephant", 2 : "Chicken", 0 : "Dog"} ##This numbers may change after  clustering! Since label of each class changes!

In [ ]:
##Test Clustering Result. Change example with 'data' variable.It goes up to 9999 if you did upload image tensor (dataset variable). Classes probabilities depends on the labeling. They may change after each clustering operation.
data = 1568
plt.imshow(dataset[data])
print(f'Chicken Prob. = {round(weights[data][2],3)} \nButterfly Prob. = {round(weights[data][3],3)} \nDog Prob. = {round(weights[data][0],3)} \nElephant Prob. = {round(weights[data][1],3)} \nHorse Prob. = {round(weights[data][4],3)}\n')
print(animal_dict[kmeans.labels_[data]])


In [ ]:
#Split all classes to use equal number of images from each classes 
image_index = []
dog_img = np.array(img_array[8])
dog_count = 1
chicken_img = np.array(img_array[3])
chicken_count = 1
elephant_img = np.array(img_array[0])
elephant_count = 1
horse_img = np.array(img_array[1])
horse_count = 1
butterfly_img = np.array(img_array[4])
butterfly_count = 1 
for i in range(0,img_array.shape[0]) :
  if kmeans.labels_[i] == 2 and dog_count <= 1000 : ## It means it is dog
    dog_count += 1
    dog_img = np.vstack((dog_img, img_array[i]))
    image_index.append(i)
  if kmeans.labels_[i] == 0 and chicken_count <= 1000: ## It means it is chicken
    chicken_img = np.vstack((chicken_img,img_array[i]))
    chicken_count += 1 
    image_index.append(i)
  if kmeans.labels_[i] == 1 and elephant_count <= 1000: ## Elephant
    elephant_img = np.vstack((elephant_img, img_array[i]))
    elephant_count += 1
    image_index.append(i)
  if kmeans.labels_[i] == 3  and horse_count <= 1000: ## horse :
    horse_img = np.vstack((horse_img, img_array[i]))
    horse_count += 1
    image_index.append(i)
  if kmeans.labels_[i] == 4 and butterfly_count <= 1000: ## butterfly
    butterfly_img = np.vstack((butterfly_img, img_array[i]))
    butterfly_count += 1
    image_index.append(i)
print('Done!')

In [ ]:
# Concatenate animal feature vectors and shuffle them.
final_arr = np.concatenate((dog_img,chicken_img,elephant_img,horse_img,butterfly_img))
np.random.shuffle(final_arr) # Shuffle the input
print(final_arr.shape)

In [ ]:
pca_5000 = PCA(n_components=3) ## Visualize the animal feature vectors in 3D space.
pca_5000.fit(final_arr)
transform_fina = pca_5000.transform(final_arr)
kmeans_5000 = KMeans(n_clusters= 5 ,n_jobs=-1, random_state=22, max_iter= 500)
kmeans_5000.fit(transform_fina)
fig = plt.figure(figsize=(10, 6))
# Create 3D container of 
ax = plt.axes(projection = '3d')
# Visualize 3D scatter plot
v = np.zeros((final_arr.shape[0],4))
v[:,:3] = transform_fina
v[:,3] = kmeans_5000.labels_
df = pd.DataFrame(v, columns=['Feature1', 'Feature2','Feature3', "Cluster"])
ax.scatter(transform_fina[:,0],transform_fina[:,1],transform_fina[:,2], marker="o", c=df["Cluster"], s=40, cmap="RdBu")
# Give labels
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
# Save figure
plt.savefig('3d_scatter.png', dpi = 400);

In [ ]:
#Reduce feature vector dimension. Cluster them use their labels in the classifier chain.
scaled = StandardScaler().fit(final_arr)
scaled_final = scaled.transform(final_arr)
pca_final = PCA(n_components = 50)
pca_final.fit(scaled_final)
print(np.sum(pca_final.explained_variance_ratio_))
compressed_final = pca_final.transform(scaled_final)
print(compressed_final.shape)
kmeans_final = KMeans(n_clusters= 5 ,n_jobs=-1, random_state=22, max_iter= 500)
kmeans_final.fit(compressed_final)
## ONE HOT ENCODING OF LABELS
a = np.array(kmeans_final.labels_)
labels = np.zeros((a.size, a.max()+1))
labels[np.arange(a.size),a] = 1


In [ ]:
##Download classifier chain from google drive
!gdown --id 1-8jbTv79KtKVNoOozi2oW-V9zsPvR6Wr
chain = joblib.load('chain.pkl') ##Optimized classifier chain model.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalize(compressed_final), labels, random_state=0, test_size = 0.1)
a = chain.predict_proba(X_test)
data = 1561 ##Change this to see a different example 
feat_res_1 = normalize(pca_final.transform(m.predict(resize(dataset[data],size=[224,224])[np.newaxis,...])))
plt.imshow(dataset[data])
print(chain.predict_proba(feat_res_1))

Difference between two lines below, is the 'order' input which, changes  the ordering of the classes in the chain. I found the optimal ordering as Dog-Horse-Elephant-Chicken-Butterfly, but there are methods to find this optimal order automatically. 


In [ ]:
##Order is random! Performance on unseen data differs by a lot!

#X_train, X_test, Y_train, Y_test = train_test_split(normalize(compressed_final), labels, random_state=0, test_size = 0.1)
#base_lr = LogisticRegression(solver='lbfgs', random_state=1)
#chain = ClassifierChain(base_lr, order='random', random_state=1)
#chain.fit(X_train, Y_train).predict(X_test)
#a = chain.predict_proba(X_test)

In [ ]:
##Order is given to the function. Dog-Horse-Elephant-Chicken-Butterfly (Order of the Chain) Max Performance! Use ready to use model. 
#Because labeling changes after clustering above. Outputs may not match otherwise.

#X_train, X_test, Y_train, Y_test = train_test_split(normalize(compressed_final), labels, random_state=0, test_size = 0.1)
#base_lr = LogisticRegression(solver='lbfgs', random_state=1)
#chain = ClassifierChain(base_lr, order=[4,3, 2,0,1], random_state=1)
#chain.fit(X_train, Y_train).predict(X_test)
#a = chain.predict_proba(X_test)

In [ ]:
## This Function is for the performance change of the chain when the ordering differs.
## Note that these results are on the trained class. Like I said, unseen data
##Performance completely different from the graph. You can see it in the report.g
base_lr = LogisticRegression()
ovr = OneVsRestClassifier(base_lr)
ovr.fit(X_train, Y_train)
Y_pred_ovr = ovr.predict(X_test)
ovr_jaccard_score = jaccard_score(Y_test, Y_pred_ovr, average='samples')
# Fit an ensemble of logistic regression classifier chains and take the
# take the average prediction of all the chains.
chains = [ClassifierChain(base_lr, order='random', random_state=i)
          for i in range(10)]
for each_chain in chains:
    each_chain.fit(X_train, Y_train)
Y_pred_chains = np.array([each_chain.predict(X_test) for chain in
                          chains])
chain_jaccard_scores = [jaccard_score(Y_test, Y_pred_chain >= .5,
                                      average='samples')
                        for Y_pred_chain in Y_pred_chains]

Y_pred_ensemble = Y_pred_chains.mean(axis=0)
ensemble_jaccard_score = jaccard_score(Y_test,
                                       Y_pred_ensemble >= .5,
                                       average='samples')

model_scores = [ovr_jaccard_score] + chain_jaccard_scores
model_scores.append(ensemble_jaccard_score)

model_names = ('Independent',
               'Chain 1',
               'Chain 2',
               'Chain 3',
               'Chain 4',
               'Chain 5',
               'Chain 6',
               'Chain 7',
               'Chain 8',
               'Chain 9',
               'Chain 10',
               'Ensemble')

x_pos = np.arange(len(model_names))

# Plot the Jaccard similarity scores for the independent model, each of the
# chains, and the ensemble (note that the vertical axis on this plot does
# not begin at 0).

fig, ax = plt.subplots(figsize=(7, 4))
ax.grid(True)
ax.set_title('Classifier Chain Ensemble Performance Comparison')
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, rotation='vertical')
ax.set_ylabel('Jaccard Similarity Score')
ax.set_ylim([min(model_scores) * .9, max(model_scores) * 1.1])
colors = ['r'] + ['b'] * len(chain_jaccard_scores) + ['g']
ax.bar(x_pos, model_scores, alpha=0.5, color=colors)
plt.tight_layout()
plt.show()

##Classifier Chain Randomly Select "starting class" and in other words "ordering" of classes, therefore
##Their order also changes the performance of the classifier chain.
##In this example though there is not much difference between different class order!
##BUT, when system is introduced with "unseen" data, the responses will differ by a lot!

In [ ]:
 ##Percentage on the test set that alert the system where they should not have! (out of 499)
threshold = 0.80
repetitive = 0
for results in a : 
  res = np.amax(results)
  if res < threshold :
    repetitive += 1
print(f'{round((100 - repetitive/499 * 100),2)}% accuracy on the test set')

In [ ]:
!gdown --id 1UaoB4Ajna-FW3YtnrXcQByi464DiaDr3
!!unzip 'Test_Images-zipped.zip'

In [ ]:
### Test on Specific Animal Species.

data = []
drive_path = 'Test_Images/Random Animal Images/'
net_sum = 0
file_number = 0
under_threshold = 0
above_threshold = 0
for filename in os.listdir(drive_path):
    if filename.endswith("jpg"): 
        file_number += 1
        # Your code comes here such as 
        feat_res_1 = normalize(pca_final.transform(m.predict(resize((imread(drive_path+filename)/255.),[224,224])[np.newaxis,...])))
        probabilities = chain.predict_proba(feat_res_1)
        res = np.amax(probabilities)
        net_sum += res
        #print(res)
        if res < threshold :
          under_threshold += 1
        else :
          above_threshold += 1
print(f' System detected new species with accuracy of {100 * (under_threshold/file_number)}')

In [ ]:
### Test on whole test images dataset taken from google images.

data = []
drive_path = 'Test_Images/'
net_sum = 0
file_number = 0
under_threshold = 0
above_threshold = 0
for filename in os.listdir(drive_path):
  for filename_1 in os.listdir(drive_path+filename) :
    if filename_1.endswith("jpg"): 
        file_number += 1
        feat_res_1 = normalize(pca_final.transform(m.predict(resize((imread(drive_path+filename+"/"+filename_1)/255.),[224,224])[np.newaxis,...])))
        probabilities = chain.predict_proba(feat_res_1)
        res = np.amax(probabilities)
        net_sum += res
        if res < threshold :
          under_threshold += 1
        else :
          above_threshold += 1
print(f' System detected new species with accuracy of {100 * (under_threshold/file_number)}')

In [ ]:
animal_dict = {3 : "Dog", 4 : "Horse", 2 : "Elephant", 0 : "Chicken", 1 : "Butterfly"} ##This is the label dictionary for ready 2 use scikit model I trained 

In [ ]:
image_url = 'https://www.diyadinnet.com/d/ruya/ruyada-at-gormek-nedir-siyah-beyaz-ata-binmek-ne-anlama-gelir-3280.jpg' #Test Horse Class
#image_url = 'https://media.tacdn.com/media/attractions-splice-spp-674x446/09/19/7c/5f.jpg'  #Test Elephant Class
#image_url = 'https://media.tacdn.com/media/attractions-splice-spp-674x446/06/e4/c9/d4.jpg'  #Test Butterfly Class
#image_url = 'https://morningchores.com/wp-content/uploads/2018/12/Everything-You-Should-Consider-for-Solid-Chicken-Health-Management-FI.jpg' #Test Chicken class
#image_url = 'https://upload.wikimedia.org/wikipedia/commons/2/23/Canis_lupus.jpg' #Test Dog Class or Unseen Species.
feat_res = normalize(pca_final.transform(m.predict(resize((imread(image_url)/255.),[224,224])[np.newaxis,...])))
result = (chain.predict_proba(feat_res))
max_perc = np.amax(result)
max_index = np.argmax(result)
##Dogs have to be handled separately. Because there are a lot of dog species out there and
## Dataset does not contain much of these species most of them labrador or golden.
## I found out after the checking the system with couple of different dog species. Works quite well.
if max_perc < threshold and max_index != 3  :
 print(f"Max similarity percentage is {round(max_perc * 100,2)}% and system says it can be a {animal_dict[max_index]}. Percentage is not high enough though. This might be a new species. Let me Zero-Shot it...")
 imshow(image_url)

elif max_perc > 60 and max_index == 3 : ## 3 is dog index.
  print(f"I know this species! It is a {animal_dict[max_index]} with probability of {round(max_perc * 100,2)}%")
  imshow(image_url)
else :
  print(f"I know this species! It is a {animal_dict[max_index]} with probability of {round(max_perc * 100,2)}%")
  imshow(image_url)

In [ ]:
##BELOW IS RELATED ZERO SHOT  / Speculation Part

In [ ]:
# !brew install wget ##Word2Vec Do not try to use and extract these here. Session will crush due to excessive use of RAM!

# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
!gdown --id 14za0w3HRSb1IQRq6sUzbLYHLJcaMX3Mx ##Zebra.npy
!gdown --id 1WqKZfikA3t5FyIlkZ_rU3gH4LQEvzMsl ##Wolf.npy
!gdown --id 1wlDoWtSls4aPRiT2t3eB55VgZkUCuH9z ##Spider.npy
!gdown --id 1WxII7q0XAabcoUHjscEx_sstQKUuuRT7 ##Human.npy
!gdown --id 1EmJZ0nsgosEKpHncl8xPSSlV-IG7ZPUU ##Horse.npy
!gdown --id 1K9FPPXj-Tw_tzGNrOAkBfF1kI7OEOcOX ##Giraffe.npy
!gdown --id 1N4PTE5AGi24NxgkBg--I00r7nNzc-W4W ##Fox.npy
!gdown --id 1Ue6hzoygIW_W7k71pMtXTPsbZR9cxvzX ##Elephant.npy
!gdown --id 1z0Ng0LTfhZitW3dsjM8-Whgq0YFJ-BaA ##Dog.npy
!gdown --id 1qhPZCmZDrdQwbpdKYLIlKysngzFr_PSJ ##Chicken.npy
!gdown --id 1FbqtVRYm-aDQ4Sy2RCaxuBDeydWkAVY6 ##Cat.npy
!gdown --id 1wIBciukjDb8yDoPFbgWs5VZrF7ehRh1I ##butterfly.npy
!gdown --id 10krVBYq_gppAmKxEK7kGLLQ_k4nvDufo ##Bear.npy
print('Done!')

In [ ]:
##Zero-Shot and Training Class Word2Vec Representation. Each of them is a (300,) vector
##It is almost impossible with only having 12 gb ram and upload whole word2vec in this environments.
##Therefore i downloaded the word2vec in my local computer and just got what I needed for this model.

'''
ZERO-SHOT CLASSES :
BEAR ZEBRA GIRAFFE SPIDER HUMAN FOX WOLF CAT

TRAINING CLASSES :
DOG HORSE ELEPHANT BUTTERFULY CHICKEN
'''

dog = np.load('dog.npy') 
horse = np.load('horse.npy')
elephant = np.load('elephant.npy')
butterfly =np.load('butterfly.npy')
chicken = np.load('chicken.npy')
bear = np.load('bear.npy')
zebra = np.load('zebra.npy')
giraffe = np.load('giraffe.npy')
spider = np.load('spider.npy')
human = np.load('human.npy')
fox = np.load('fox.npy')
wolf = np.load('wolf.npy')
cat = np.load('cat.npy')

In [ ]:
pca_clust = PCA(n_components = 30)
pca_clust.fit(scaled_array)
print(np.sum(pca_clust.explained_variance_ratio_))
compressed_zsl = pca_clust.transform(scaled_array)
print(compressed_zsl.shape)
#kmeans_zsl = KMeans(n_clusters= 5 ,n_jobs=-1, random_state=22, max_iter= 500)
#kmeans_zsl.fit(compressed_zsl)

In [ ]:
!gdown --id 1-OP1f9mWWpiYEJ4D6q4EhabomRPLOwxa ##Download clustering result from drive ('kmeans_save.pkl')

In [ ]:
load_kmeans = pickle.load(open("kmeans_save.pkl", "rb")) ## Load clustering result

In [ ]:
animal_dict = {4 : "Dog", 3 : "Horse", 0 : "Elephant", 2 : "Chicken", 1 : "Butterfly"} ##This is the label dictionary for ready 2 use scikit model I trained 

In [ ]:
data = 23
plt.imshow(dataset[data])
print(f'Chicken Prob. = {round(weights[data][2],3)} \nButterfly Prob. = {round(weights[data][1],3)} \nDog Prob. = {round(weights[data][4],3)} \nElephant Prob. = {round(weights[data][0],3)} \nHorse Prob. = {round(weights[data][3],3)}\n')
print(animal_dict[load_kmeans.labels_[data]])

In [ ]:
semantic_arr = np.array(elephant) ##First image was an elephant. To initialize semantic array, i directly used elephant semantic vector.
##Create Semantic Array to Map onto. For zero-shot purposes. 
for i in range(1,kmeans.labels_.shape[0]) :
  if kmeans.labels_[i] == 0 : ##It means it is elephant
    semantic_arr = np.vstack((semantic_arr, elephant))
  elif kmeans.labels_[i] == 1 : ##It means it is butterfly
    semantic_arr = np.vstack((semantic_arr, butterfly))
  elif kmeans.labels_[i] == 2 : ## It means it is chicken
    semantic_arr = np.vstack((semantic_arr, chicken))
  elif kmeans.labels_[i] == 3 : ##It means it is horse
    semantic_arr = np.vstack((semantic_arr, horse))
  elif kmeans.labels_[i] == 4 : ##It means it is dog
    semantic_arr = np.vstack((semantic_arr, dog))
print(semantic_arr.shape)
print('Done!')

In [ ]:
def  build_model():
    model = Sequential()
    model.add(Dense(2048, input_shape=(300,), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(300, activation='relu'))
#    model.add(Dense(NUM_CLASS, activation='softmax', trainable=False, kernel_initializer=custom_kernel_init))

    print("-> model building is completed.")
    return model
def early_stopping() :
  return EarlyStopping(
      monitor = 'val_accuracy',
      patience = 20
  )
def get_checkpoint(): 
  return ModelCheckpoint(
    '/content/drive/MyDrive/Me536_Final_Project/Zero Shot Learning/zsl_best_corrected_semantic_array', monitor='val_accuracy', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq="epoch",
    options=None
)
##Training parameters of Zero-shot model


#adam = Adam(lr=5e-5)
#normalized_compressed = normalize(compressed_vector)
#zsl_model = build_model()
#zsl_model.compile(loss      = 'mean_squared_error',
#                  optimizer = adam,
#                  metrics   = ['accuracy'])


In [ ]:
!gdown --id 1GqgIs4KdjErBW7QPSMUhDs1qcmRnk69s ##Download zero-shot model from drive
!unzip 'ZSL_Model_Another_zipped.zip' ##Unzip 
zsl_model = load_model('ZSL_Model_Another') ##Load the already trained zero-shot model

In [ ]:
zsl_model.summary()

In [ ]:
class_dict = {"dog" : dog,
     "elephant" :elephant,
     "horse" : horse, 
     "chicken" : chicken, 
     "butterfly" : butterfly, 
     "bear" : bear,
     "zebra" : zebra, 
     "giraffe" : giraffe, 
     "spider" : spider, 
      "human" : human, 
      "fox" : fox, 
      "wolf": wolf,
      "cat" : cat }

In [ ]:
pca_zsl = PCA(n_components = 300) ##Reduce the dimension of input to the trained model input size.
pca_zsl.fit(scaled_array)

In [ ]:
path = 'https://icatcare.org/app/uploads/2018/06/Layer-1704-1920x840.jpg' ##Image path
feat_res_1 = normalize(pca_zsl.transform((m.predict(resize((imread(path)/255.),[224,224])[np.newaxis,...])))) #Resize, get the feature vector from the trained model, reduce its dimension and normalize the input
img_res_1 = zsl_model.predict(feat_res_1) #Get the prediction from the Zero-shot model
img_res_1 = img_res_1.reshape(300,)
distance_list_another = list()
for word in class_dict :
  x = class_dict[word]
  dist = np.linalg.norm(x - img_res_1)
  distance_list_another.append([dist, word])
distance_list_another = sorted(distance_list_another, key = lambda x : x[0])
print("Top 5 predictions are :")
for i in range(0,5) : ##Top 3 Class
  print(f'{i+1} {distance_list_another[i][1]}')
plt.imshow(imread(path))